In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from hidden import *       # Функции, реалзация которых не особо интересна в свете этой темы
from answers import *      # Ответы
from visual_utils import * # Функции визуализации
from cg_common import * # Должно быть перенесено в общую либу
import numpy as np

# Ушной метод триангуляция многоугольников

В теме посвященной [трингуляции монотонного многоугольника(будет ссылка на конспект Лёни)]() мы доказали существование триангуляции у простого $n$-вершинного многоугольника и собственно разобрали один из способов ее нахождения.
Здесь мы рассмотрим другой довольно интересный, а так же простой в понимании и реализации алгоритм триангулирования многоугольников, называющийся **ушной триангуляцией**.  

### Определение
Три последовательные вершины $v_{i-1}$, $v_{i}$, $v_{i+1}$ многоугольника $P$ называются **ухом**, если треугольник образованный этими вершинами не содержит внутри себя и на границе остальных вершин многоугльника $P$, а так же угол при вершине $v_{0}$, называемой **вершиной уха**, строго меньше $\pi$ (см рис.1). 

![определение уха](./img/def_ear.svg)

#### Теорема 1 (О существовании двух ушей у многоугольника)
> У любого простого $n$-вершинного многоугольника $P$ всегда существуют два непересекающихся между собой уха.

$\triangleright$
<div style="padding-left:40px">
Доказывать будем по индукции по количеству $n$ вершин.
База: $n = 4$.
![База](./img/th1_base.svg)

Предположим, что для всех многоугольников, количество вершин в которых не больше $n$, теорема верна.
Рассмотрим многоугольник $P$, в котором $n + 1$ вершина.
Далее возможны два случая:
* Произвольная выпуклая вершина $v_{i}$ многоугольника $P$ является ухом.
Отрезав это ухо, мы уменьшим число вершин $P$ на одну.
В результате, получим $n$-вершинный многоугольник $P'$.
По предположению индукции у него существует два непересекающихся уха.
Учитывая, что уши $P'$ являются и ушами $P$, можно утверждать, что для $P$ теорема верна.
![Ухо](./img/th1_case_1.svg)

* Произвольная выпуклая вершина $v_{i}$ многоугольника $P$ не является ухом.
В таком случае в треугольнике $\Delta v_{i-1}v_{i}v_{i+1}$ лежит хотя бы одна отличная от вершин треугольника вершина из $P$.
Если их несколько, то выберем вершину $q$, которая будет ближе всего к $v_{i}$.
Проведём отрезок $v_{i}q$, который разделит $P$ на два многоугольника: $P_{1}$ и $P_{2}$.
В каждом из них будет не более $n$ вершин, следовательно у каждого будет по два непересекающихся уха.
Даже если предположить, что ухо из $P_{1}$ и ухо из $P_{2}$ будут пересекаться по стороне $v_{i}q$, в $P$ всё равно будет не менее двух непересекающихся ушей.
![Не ухо](./img/th1_case_2.svg)
</div>
$\triangleleft$

### Задача
>Пусть нам дается список вершин простого $n$-вершинного многоугольника, задающий их в порядке обхода против часовой стрелки. Надо затриангулировать этот многоугольник.

### Алгоритм
Идея заключается в последовательном отрезании ушей от многоугольника, пока он не станет треугольником.
Зафиксируем стартовую вершину многоугльника и будем обходить его в поиске ушей, добавляя в стек $S$ вершины, обработанные нами и не являющиеся ушами.
Ломаная, построенная на вершинах из стека, будет отделять отрезанные уши от еще не просмотренных вершин многоугольника. 

Когда при обходе мы находим вершину уха, то отрезаем ее, добавляя диагональ, и переходим к предыдущей вершине из стека.
Если она стала вершиной уха, то отрезаем ее и дальше спускаемся по стеку.
Этот процесс остановится, когда мы либо достигнем стартовой вершины (стек пуст), либо текущая вершина перестанет быть ухом.
Тогда мы переходим к вершине, являющейся концом последней добавленной диагонали, и продолжаем обход многоугольника в поиске ушей.

В итоге, мы придем к тому, что от нерассмотренной части многоугольника останется только одно ребро, а остальная часть будет затриангулирован.  

### Ушная проверка
Для того чтобы быть ухом, вершина прежде всего должна быть выпуклой, проверки этого, удобно использовать [левый поворот](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9F%D1%80%D0%B5%D0%B4%D0%B8%D0%BA%D0%B0%D1%82_%22%D0%BB%D0%B5%D0%B2%D1%8B%D0%B9_%D0%BF%D0%BE%D0%B2%D0%BE%D1%80%D0%BE%D1%82%22).
Затем нужно проверить, что никакие вершины многоугольника не лежат внутри или на границе треугольника, соответствующего текущей предполагаемой вершине уха.

Далее вам предлагается потренироваться в решении этой задачи, дополнив следующую функцию.

In [2]:
def is_outer(v0,v1,v2,v):
    '''Проверяет, лежит ли вершина строго снаружи трегольника 
    На вход подаются три вершины v0, v1 и v2, образующие потенциальное ухо, а также вершина
    многоугольника принадлежность которой ножо проверить. Вершины имеют тип Vertex (эти классы
    определены в hidden.py). У них есть поля x, y. Вам также доступна функция turn - предикат 
    левого поворота, которая принимает три Vertex.
    Требуется вернуть True, если v строго снаружи тегольника, иначе False.
    '''
    
    return is_outer_answer(v0,v1,v2,v) # Заглушка с функцией-ответом на задачу

#### Лемма 1 (О нерассмотренных вершинах многоугольника)
> Проверять на принадлежность уху нужно только те вершины многоугольника, которые еще не были рассмотрены.

$\triangleright$
<div style="padding-left:40px">
Если в треугольник попали и рассмотренные, и нерассмотренные вершины, то по наличию первого типа вершин мы поймем, что это не ухо.
Пусть в трегольник попали только нерассмотренные вершины (см.рис. 5), выберем из них самую ближайшую к $v$, назовем ее $w_{i}$.
Рассмотрим многоугольник образованный цепочкой вершин $w_{i}, w_{i + 1},\dots, w_{k}, v, w_{i}$.
По теореме 1 у него существуют два непересекающихся уха, так что даже если предположить, что одно из них образовано $v$ или $w_{i}$, то есть второе, которое соответствует вершине из стека $S$, а это противоречит инварианту стека, по которому вершины в $S$ не являются ушами.
![рис. 5](img/lemma_1.svg)
</div>
$\triangleleft$

Теперь соберем полученные факты вместе и напишем полноценную ушную проверку.

In [3]:
def is_ear(hedge, probable_ear, pred):
    '''Проверяет, образует ли треугольник ухо
    На вход подается: экземпляр класса Hedge - ребро, соответствующее вершине, с которой надо 
    начинать проверку на принадлежность; массив из трех последовательных вершин, образующих 
    потенциальное ухо; предикат, задающий нерассмотренные вершины, который мы определим 
    немного позже в соответствии с леммой 1.
    
    Возвращает True, если probable_ear действительно образует ухо, иначе False.
    '''
    
    while pred(hedge):
        if not is_outer(*probable_ear, hedge.origin):
            return False 
        hedge = hedge.next
    return True 

А теперь давайте напишем сам алгоритм.
Воспользуемся фактом, что триангуляция $n$-вершинного многоугольника содержит $n - 3$ диагонали (доказанном в теме монотонной трианглуляции) как признак завершения алгоритма.

In [8]:
# Примеры многоугольников
examples = {
        'some': [(4, 0), (2, 1), (3, 6), (0, 5), (-1, 6), (-3, 4), (-1, 2), (-2, 0),
              (-4, 1), (-5, -2), (-2, -4), (0, -3), (2, -4), (1, -1), (3, -2)]
        ,
        'square': [(0,0), (1,0), (1,1), (0,1)]
        ,
        'arrow': [(0,0), (2,3), (4,0), (2,6)]
        ,
        'star': [(0,0), (7,1), (7,3), (6,5), (5,6), (3,7), (1,7)]
        ,
        'substar': [(0,0), (1,6), (7,1), (7,3), (6,5), (5,6), (3,7), (1,7)]
        ,
        'hook': [(0,3), (2,4), (3,3), (3,2), (2,1), (2,2), (2,3), (1,2), (1,1), (3,0), (4,4), (2,5), (1,4), (0,5)]
        }

# Вершины многоугольника в порядке обхода против часовой стрелки
source = examples['hook']
# Преобразовываем пары в наш класс
source = list(map(lambda i: Vertex(i[0], i[1]), source))
# Многоугольник хранится в DCEL
D = build_dcel(source)
# Добавленные диагонали будем хранить в отдельном списке для наглядной визуализации
D1 = []
# Стек вершин
S = []   

In [11]:
D1 = []
S = []

In [12]:
def break_into_pieces_triang(folder='.ear_clipping'):
    # С помощью этой функции мы будем делать "снимки" состояния алгоритма, чтобы потом
    # можно было посмотреть его по шагам.
    dump = create_dump_func(folder, visual_dump_ear_clipping_triangulation, D, D1, S)
    
    def peek(s):
        return s[len(s) - 1]
    
    start = D[0]
    dump(start.origin)
    S.append(start)    

    while len(D1) != len(D) - 3:
        cur = peek(S).next
        while True:
            # At the start point 
            if len(S) == 0:
                dump(cur.origin)
                if len(D1) == len(D) - 3:
                    break
                S.append(cur)
                cur = cur.next
                
            v0 = peek(S).origin
            v1 = cur.origin
            v2 = cur.next.origin
            
            if turn(v0, v1, v2) >= 1 and is_ear(cur.next.next, probable_ear=[v0, v1, v2],
                                                pred=lambda hedge: hedge != start):
                dump(cur.origin)

                prev = S.pop()                
                d = add_diagonal_with_next(prev, cur.next)
                D1.append(d)
                dump(cur.origin)

                cur = d
            else:
                dump(cur.origin)
                S.append(cur)
                cur = cur.next
                break
                
    # Remove last edge from of last triangle
    S.clear()
    dump(None)

In [13]:
# Запуск алгоритма
break_into_pieces_triang()

Можно проследить за алгоритмом по шагам, используя виджет ниже.
У исходного многоугольника рёбра чёрные.
Рёбра, образованные вершинами из стека и текущей вершиной - красные.
А диагонали разбиения проведены синим пунктиром.

In [14]:
from IPython.display import display
display(SlideShower('.ear_clipping'))

#### Доказательство корректности
Доказательство корректности алгоритма достаточно легко вывести из его построения.
Отрезаемые уши попарно непересекаются, а следовательно и диагонали попарно не пересекаются.
Отрезая ухо мы добавляем диагональ, которая сохраняет корректность многоугольника, при этом уменьшая количество вершин в нем.
В итоге многоугольник становится треугольником, который по определению затриангулирован, а все вершины соединены диаголнялями.

#### Оценка времени работы
Каждая вершина бывает в стеке только один раз.
На каждом шаге алгоритма мы либо отрезаем ухо и спускаемя по стеку, либо переходим к следующей вершине многоугольника.
Для каждой добавленной вершины в начинающейся с нее цепочке последовательных отрезаний ушей мы можем зафейлить ушную проверку только один раз, после чего добавим следующую вершину полигона в стек.
Т.е. полный такой обход полигона занимает $O(n)$ времени.
Ушная проверка для одной вершины занимает $O(n)$ времени.
А тогда общая оценка на время $O(n^2)$.

#### Оценка на память
Для хранения самого полигона мы используем DCEL, который занимает линейное количество памяти.
И заводим стек для хранения диагоналей, которых $n-2$.
Таким образом, потребляемое количество памяти линейно.

Итого, мы получили следующий результат:
> Простой многоугольник с $n$ вершинами может быть затриангулирован за $O(n^2)$ времени, используя $O(n)$ памяти. 

### Триангуляция многоугольника с дырками

Две вершины называются **взаимно видимыми**, если соединяющая их диагональ не пересекает ребра многоугольника и не содержит вершин из него.

![Взаимно видимые вершины](img/def_visible.svg)

Пусть теперь мы хотим затриангулировать полигон, в котором есть дырки.
Для начала предположим, что она одна.
Сведем задачу к триангуляции без дырок.
Идея заключается в том, чтобы соединить **взаимно видимые** вершину внешнего полигона и вершину внутренегом(задающего дырку) полигона.
Пусть вершина $v_{i}$ принадлежит внешнему, а $u_{j}$ - внутреннему.
Добавив две новые вершины $v'$ и $u'$, ребра $v'u'$ и $uv$, сделаем наш многоугольник простым.
А его мы уже умеем триангулировать.

Однако как мы найдем эти самые взаимно видимые вершины? 

### Алгоритм поиска взаимно видимых вершин
Найдем самую левую точку внутреннего полигона, а из таких самую нижнюю, назовем ее $p$.
Проведем из нее луч в минус бесконечность по оси $Ox$.
Он пересечет какие-то ребра или вершины внешнего полигона.
Возьмем из таких точек пересечения самую правую, назовем ее $c$.
Возможны два случая:
* Если это вершина, то мы нашли подходящую видимую вершину внешнего полигона (см. рис. 7а).
* Иначе мы попали во внутренную область ребра $e'e$ (см. рис. 7б).
Тогда соединим его нижний конец $e$ и $p$.
Рассмотрим невыпуклые вершины внешнего полигона, которые попали в треугольник $\Delta epc$.
Отсортируем их по углу, образованному с отрезком $pc$.
Возьмем минимальную, а среди таких самую правую.

![рис 7](img/algo_hole.svg)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from hidden import *       # Функции, реалзация которых не особо интересна в свете этой темы
from answers import *      # Ответы
from visual_utils import * # Функции визуализации
from cg_common import * # Должно быть перенесено в общую либу
import numpy as np

In [11]:
polygon = list(map(lambda i: Vertex(i[0], i[1]),
        [(8, 18), (2, 12), (1, 14), (2, 11), (6, 15), (3, 3), (5, 1), (4, 5),(5, 6),
         (6, 8), (8, 4), (8, 7), (7, 8), (9, 9), (9, 10), (13, 8), (18, 13), (17, 15)]))
# Точки "дырок" мы задаём уже по часовой стрелке - многоугольник снаружи них, а не внутри
hole1 = list(map(lambda i: Vertex(i[0], i[1]),
        [(15, 15), (12, 14), (14, 13), (12, 12), (14, 10), (15, 13)])) 
hole2 = list(map(lambda i: Vertex(i[0], i[1]),
        [(11, 16), (8, 15), (9, 13)]))

In [4]:
class Ungle:
    def __init__(self, vertex, angular_point):
        self.vect = vertex
        self.angular_point = angular_point 
        
    def __lt__(self, other):
        t = turn(self.vect, self.angular_point, other.vect) 
        return t < 0 or t == 0 and self.vect.x > other.vect.x

In [5]:
def is_outer(v0,v1,v2,v):
    '''Проверяет, лежит ли вершина строго снаружи трегольника 
    На вход подаются три вершины v0, v1 и v2, образующие потенциальное ухо, а также вершина
    многоугольника принадлежность которой ножо проверить. Вершины имеют тип Vertex (эти классы
    определены в hidden.py). У них есть поля x, y. Вам также доступна функция turn - предикат 
    левого поворота, которая принимает три Vertex.
    Требуется вернуть True, если v строго снаружи тегольника, иначе False.
    '''
    
    return is_outer_answer(v0,v1,v2,v) # Заглушка с функцией-ответом на задачу

In [6]:
def merge_polygons(outer_polygon, outer_h, inner_polygon, inner_h):
    outer_prev = outer_h.prev
    inner_prev = inner_h.prev
    
    def add_edge(from_h, to_h):
        # новое ребро
        h = Hedge(Vertex(from_h.next.origin.x, from_h.next.origin.y))
        h.twin = Hedge(Vertex(to_h.origin.x, to_h.origin.y))
        h.twin.twin = h

        # from -> new
        from_h.next = h
        h.prev = from_h  
        from_h.twin.prev = h.twin
        h.twin.next = from_h.twin

        # new -> to
        h.next = to_h
        to_h.prev = h
        h.twin.prev = to_h.twin
        to_h.twin.prev = h.twin
        return h
    
    h1 = add_edge(outer_prev, inner_h)
    h2 = add_edge(inner_prev, outer_h)
    
    outer_polygon.append(h1)
    outer_polygon.append(h2)
    for hedge in inner_polygon:
        outer_polygon.append(hedge)
    inner_polygon.clear()
        
    return outer_polygon

In [16]:
polygon = list(map(lambda i: Vertex(i[0], i[1]),
        [(3, 1), (0, 3), (-4, 2), (-2, 0), (-3, -1), (-2, -7), (1, -3), (3, -4), (4, -1)]))
# Точки "дырок" мы задаём уже по часовой стрелке - многоугольник снаружи них, а не внутри
hole1 = list(map(lambda i: Vertex(i[0], i[1]),
        [(0, 1), (1, 0), (0, -2), (-1, 0)])) 
hole2 = list(map(lambda i: Vertex(i[0], i[1]),
        [(-1, -2), (-1, -5), (-2, -3)]))

In [17]:
outer = build_dcel(polygon)
inner1 = build_dcel(hole1)
inner2 = build_dcel(hole2)

D = merge_polygons(outer, outer[0], inner1, inner1[0])
D = merge_polygons(outer, outer[4], inner2, inner2[0])

In [18]:
def break_into_pieces_bypass_forward(folder='.bypass_forward'):
    dump = create_dump_func(folder, visual_dump_bypass, D)
    
    start = D[0]
    dump(start.origin)
    cur = start.next
    while cur != start:
        dump(cur.origin)
        cur = cur.next
        
break_into_pieces_bypass_forward()

from IPython.display import display
display(SlideShower('.bypass_forward'))

In [ ]:
def break_into_pieces_bypass_backward(folder='.bypass_backward'):
    dump = create_dump_func(folder, visual_dump_bypass, D)
    
    start = D[0]
    dump(start.origin)
    cur = start.prev
    while cur != start:
        dump(cur.origin)
        cur = cur.prev
        
break_into_pieces_bypass_backward()

from IPython.display import display
display(SlideShower('.bypass_backward'))

In [13]:
def break_into_pieces_holes_merging(polygon, *holes, folder='.holes_merging'):
    # Лесикографический порядок на точках(костыль для точек Леонида)
    Vertex.__lt__ = lambda self, other: self.x < other.x or (self.x == other.x and self.y < other.y)
    # converter to point(VERTEX)
    tp = lambda x: Vertex(x[0], x[1])
    # converter to np.array
    tnp = lambda v: np.array([v.x, v.y]) 

    # Вместо луча в минус бесконечноть по оси Ох будем проводить
    # отрезок до координаты самой левой точки полигона. 
    min_x = polygon[0].origin.x
    for hedge in polygon:
        min_x = min(hedge.origin.x, min_x)
   
    # Отсортируем дырки по неубыванию самой левой нижней точки
    order = []
    for i, hole in enumerate(holes):
        # left_lower_hedge
        llh = hole[0]
        for h in hole:
            if h.origin < llh.origin:
                llh = h
        order.append((i, llh))

    ordered_holes = []
    for pos, llh in sorted(order, key=lambda v: v[1].origin):
        ordered_holes.append((llh, holes[pos]))
    
    # С помощью этой функции мы будем делать "снимки" состояния алгоритма, чтобы потом
    # можно было посмотреть его по шагам.
    dump = create_dump_func(folder, visual_dump_holes_merging,
                            polygon, list(map(lambda x: x[1], ordered_holes)))
    
    dump({'step':Merging_step.BEFORE_START})
    
    # По очереди сливаем дырки с полигоном
    for llh, hole in ordered_holes:
        dump({'llh':llh,'step':Merging_step.LEFT_LOWER})
        
        # Ищем ребро или вершину с максимальным по оси Ох пересечением 
        intersected = []
        a = llh.origin
        b = tp([min_x, llh.origin.y])
        dump({'ray':(a,b), 'step':Merging_step.RAY})
        for h in polygon:
            c = h.origin
            d = h.twin.origin
            if turn(a, b, c) * turn(a, b, d) <= 0 and turn(c, d, a) * turn(c, d, b) <= 0:
                ip = get_intersection_point(tnp(a), tnp(b), tnp(c), tnp(d))
                intersected.append((ip[0]/ip[2], ip[0]/ip[2], h))
        
        # Сортируем точки пересечения по убыванию
        intersected = sorted(intersected, key=lambda x: -x[0])
        dump({'ray':(a,b), 'intersected':intersected, 'step':Merging_step.INTERSECTED})
        
        # Выбираем отрезок\вершину с ближайшей к вершине дырки точкой пересечения 
        closest = intersected[0]
        v0 = a
        v1 = closest[2].origin
        v2 = closest[2].twin.origin
        
        if v1.y != v2.y and turn(a, b, v1) == 0:
            # Попали в начало ребра
            dump({'ray':(a,b), 'closest':v1, 'step':Merging_step.REFLEX_VERTEXES})
            merge_polygons(polygon, closest[2], hole, llh)
            dump({'polygon':polygon, 'step':Merging_step.MERGE})
            continue
        elif v1.y == v2.y or turn(a, b, v2) == 0:
            # Попали в конец ребра(или оно горизонтальное)
            dump({'ray':(a,b), 'closest':v2, 'step':Merging_step.REFLEX_VERTEXES})
            merge_polygons(polygon, closest[2].next, hole, llh)
            dump({'polygon':polygon, 'step':Merging_step.MERGE})
            continue
        
        # Попали в ребро
        dump({'ray':(a,b), 'closest':closest[2], 'step':Merging_step.REFLEX_VERTEXES})
        
        # Ищем выпуклые вершины, попавшие в треугольник         
        reflex_origins = [closest[2].next] # нижняя вершина ребра, которое пересек луч
        for h in polygon:
            v = h.origin
            if turn(h.prev.origin, v, h.next.origin) <= 0 and \
               v != v1 and v != v2 and not is_outer(v0,v1,v2,v):
                reflex_origins.append(h)
        dump({'triangle':(v0,v1,v2), 'reflex':reflex_origins, 'step':Merging_step.REFLEX_SORTING})
        
        # Отсортируем вершины по углу относительно отрезка ab
        reflex_origins = sorted(reflex_origins, key=lambda hedge: Ungle(hedge.origin, llh.origin))
        dump({'triangle':(v0,v1,v2), 'closest':reflex_origins[0], 'step':Merging_step.CLOSEST})

        merge_polygons(polygon, reflex_origins[0], hole, llh)
        dump({'polygon':polygon, 'step':Merging_step.MERGE})
        

In [14]:
# Запуск алгоритма
break_into_pieces_holes_merging(build_dcel(polygon),
                                build_dcel(hole1),
                                build_dcel(hole2))

Можно проследить за алгоритмом по шагам, используя виджет ниже.
У исходного многоугольника и дырок чёрные рёбра.

In [15]:
from IPython.display import display
display(SlideShower('.holes_merging'))

$\triangleright$
<div style="padding-left:40px">
</div>
$\triangleleft$